<a href="https://colab.research.google.com/github/stoiver/Tohoku/blob/master/notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tohoku Notebook

These notebooks have been designed to run in the google `colaboratory` environment, which provides a jupyter notebook environment running on a virtual machine on the cloud. To use this environment you need a google account so that your notebook can be saved on google drive. 

To start interacting with the notebook follow the 
`View in Colaboratory` link above. 

## Setup Environment

If on github, first follow the link `View in Colaboratory' to start running on google's colab environment. 

### Then ....

Run the following cell to install the dependencies and some extra code for visualising on Colaboratory.
Wait until you see the comment *(6) Ready to go* before proceeding to subsequent commands. 

The install should take less than a minute (and quicker if you have already run this earlier).

In [0]:
try:
  # On colab we can install all the packages we need via the notebook
  #
  # First download the clinic repository
  import os
  os.chdir('/content')
  
  print "(1) Install netcdf nose via pip"
  !pip -q install netCDF4 nose > /dev/null 2>&1 

  print "(2) Install gdal via apt-get"
  !apt-get -q -y install python-gdal gdal-bin > /dev/null 2>&1 

  print "(3) Download anuga_core github repository"
  !git clone -b tsunami_okada  https://github.com/stoiver/anuga_core.git  > /dev/null 2>&1 

  print "(4) Install anuga"
  os.chdir('/content/anuga_core')
  !python setup.py --quiet install  > /dev/null 2>&1 
  os.chdir('..')
  
  print "(5) Download Tohoku github repository"
  !git clone https://github.com/stoiver/Tohoku.git  > /dev/null 2>&1   
  
  print "(6) Ready to go"
 
except:
  pass

### Setup inline graphics and animation

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

## Tohoku Folder

Change into the Tohoku folder

In [0]:
os.chdir('Tohoku')

## Create Domain

In [0]:
import anuga


# Create domain
dx = dy = 1000
L = 80000
W = 80000


# Create topography
def topography(x, y):
    el = -1000
    return el


domain = anuga.rectangular_cross_domain(int(L/dx), int(W/dy), len1=L, len2=W)

domain.set_name('test')
domain.set_quantity('elevation', function=topography, location='centroids')

## Apply Okada to Stage

In [0]:
from anuga.tsunami_source.okada_tsunami import earthquake_source

x0 = 40000.0
y0 = 40000.0
depth = 15.0
strike = 0.0
dip = 15.0
length = 20.0
width = 6.0
slip = 10.0
rake = 90.0
dis3 = 0.0

ns = 1

source1 = np.array([x0, y0, depth, strike, dip, length, width,
                  slip, rake, dis3])


Ts = earthquake_source(
             source=source1,
             domain=domain,
             lon_lat_degrees=False,
             lon_before_lat=True,
             utm_zone=None,
             verbose=False)

domain.set_quantity('stage', function=Ts, location='centroids')

dplotter = anuga.Domain_plotter(domain)

print np.max(dplotter.stage), np.min(dplotter.stage)
print np.max(dplotter.elev), np.min(dplotter.elev)
print np.max(dplotter.depth), np.min(dplotter.depth)

dplotter.plot_stage_frame(vmin=-3.0, vmax=8.0)

In [0]:
# Initial condition of earthquake for multiple source

x0 = [40000.0, 40000.0]
y0 = [45000.0, 35000.0]
depth = [15.0, 15.0]
strike = [0.0, 0.0]
dip = [15.0, 15.0]
length = [10.0, 10.0]
width = [6.0, 6.0]
slip = [10.0, 10.0]
rake = [90.0, 90.0]
dis3 = [0.0, 0.0]

ns = 2


source2 = np.array([x0, y0, depth, strike, dip, length, width,
                  slip, rake, dis3]).transpose()


Ts = earthquake_source(
             source=source2,
             domain=domain,
             lon_lat_degrees=False,
             lon_before_lat=True,
             utm_zone=None,
             verbose=False)

domain.set_quantity('stage', function=Ts, location='centroids')


print np.max(dplotter.stage), np.min(dplotter.stage)
print np.max(dplotter.elev), np.min(dplotter.elev)
print np.max(dplotter.depth), np.min(dplotter.depth)

dplotter.plot_stage_frame(vmin=-3.0, vmax=8.0)